In [ ]:
import os

import matplotlib.pyplot as plt

from astropy.table import Table

%matplotlib notebook

In [ ]:
import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
from lsst.daf.persistence import Butler

In [ ]:
from deepcoadd_afw_to_astropy import read_cats

In [ ]:
repo = os.path.join(os.getenv('DR1BASE'), 'repo', 'test_dr1')
rerun = os.path.join(repo, 'rerun', 'forcedPhot')

field, tract = 'LSQ13cwp', 0
# field = 'PTF11mty'

In [ ]:
J_cat, H_cat, ref_table = read_cats(field, tract=0, repo=rerun)

snr_threshold = 5
good_color = (J_cat['J_SNR'] > snr_threshold) & (H_cat['H_SNR'] > snr_threshold)

J_cat = J_cat[good_color]
H_cat = H_cat[good_color]
ref_table = ref_table[good_color]

In [ ]:
butler = Butler(rerun)

dId = {'field': field, 'filter': 'H', 'tract': 0, 'patch': '0,0'}
calexp = butler.get('deepCoadd', dataId=dId)

In [ ]:
display = afwDisplay.getDisplay(backend='ds9')

In [ ]:
display.mtv(calexp)

In [ ]:
display.setMaskTransparency(80)
display.scale("asinh", -2, 25)

In [ ]:
print([name for name in H_cat.colnames if name[:4] == 'slot'])
X = 'slot_Centroid_x'
Y = 'slot_Centroid_y'

In [ ]:
display.erase()

In [ ]:
with display.Buffering():
    for s in ref_table:
        display.dot("o", s[X], s[Y], size=10, ctype='orange')

Find LSQ13cwp based on RA, Dec: 04:03:50.662  -02:39:18.57 (J2000)

In [ ]:
LSQ13cwp_RA_str, LSQ13cwp_Dec_str = '04:03:50.662', '-02:39:18.57'
LSQ13cwp_RA, LSQ13cwp_Dec = 60.961092, -2.655158

sn_coord = afwGeom.SpherePoint(LSQ13cwp_RA, LSQ13cwp_Dec, afwGeom.degrees)

distList = []
for s in ref_table:
    this_coord = afwGeom.SpherePoint(s['coord_ra'], s['coord_dec'], afwGeom.radians)
    angSep = sn_coord.separation(this_coord)
    distList.append(angSep)
    
distance = np.array(distList)
sn_idx = np.argmin(distance)

print("Found match: Object %d at %f arcsecs" % (sn_idx, afwGeom.radToArcsec(distance[sn_idx])))

In [ ]:
sn_ref = ref_table[sn_idx]
display.dot("o", sn_ref[X], sn_ref[Y], size=20, ctype='green')

Read in the forced-src photometry files that were built off of this same reference table to extract a lightcurve for LSQ13cwp

In [ ]:
cat_dataRefs = butler.subset(datasetType='forced_src', dataId=dId)

In [ ]:
dId = [dr.dataId for dr in cat_dataRefs][0]

In [ ]:
def extract_sn_from_dataRef(dr, idx):
    cat = dr.get().asAstropy()
    return cat[idx]

In [ ]:
for dr in cat_dataRefs:
    print(dr)
    sn_this_cat = extract_sn_from_dataRef(dr, idx=sn_idx)

In [ ]:
test_dId = {'field': 'LSQ13cwp', 'filter': 'H', 'tract': 0, 'patch': '0,0', 'seq': 'A', 'night': 20131111, 'expnum': 521}
butler.get(datasetType='forced_src', dataId=test_dId)

In [ ]:
dId

In [ ]:
cat = butler.get(datasetType='forced_src', dataId=dId)

In [ ]:
dir(cat)

In [ ]:
help(cat.extract)